In [12]:
script_sql = f''' 
-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
;
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
;
'''

### Import library

In [1]:
import pandas as pd
import os
import xlwings as xw
from pathlib import  Path

#### FT and ACCOUNT

In [5]:
import pandas as pd
import os
import xlwings as xw
from pathlib import  Path
path = os.getcwd()
for file_0 in os.listdir(os.path.join(path,'file-SQL')):
    os.remove(os.path.join(path,'file-SQL',file_0))
root_folder = 'file-Statements'
for root, dirs, files in os.walk(os.path.join(path,root_folder)):
    for dir in dirs:
        subfolder = os.path.join(root, dir)
        for file in os.listdir(subfolder):
            
            # print(os.path.join(root, dir, file))
            print(f'Loading folder {subfolder} file {file}......')

            df_sk = pd.read_excel(os.path.join(subfolder, file), skipfooter=3, header=8)
            df_sk = df_sk.loc[df_sk['Số GD'].str.startswith('FT')]
            df_sk = df_sk['Số GD'].apply(lambda x: x[:12])
            
            app = xw.App()
            wb = xw.Book(os.path.join(subfolder, file))
            account = wb.sheets[0].range("E3").raw_value


            a = list(map(lambda x: "'"+x+"'",df_sk.values))
            list_ft = ','.join(a) #string
            count = df_sk.count()
            if count < 1000:
                # with open(f'{file[:len(file)-4]}.sql', 'w', encoding= 'utf-8-sig') as sql:
                # with open(os.path.join(subfolder)+ f'//{file[:len(file)-4]}.txt', 'w', encoding= 'utf-8-sig') as sql:
                with open(os.path.join('file-SQL')+ f'//{file[:len(file)-4]}.txt', 'w', encoding= 'utf-8-sig') as sql:
                    sql.write(f'''-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
;
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
;
''')
            if count >= 1000:
                print(f'Loading file > 1000 {file}......')
                sl_file = (count // 1000) + 1
                sl_row = (count // sl_file)  + 1
                start = 1
                start_index = 0
                end_index = sl_row
                while start <= sl_file:
                    new_list_a = a[start_index:end_index]
                    list_ft = ','.join(new_list_a)
                    # with open(f'{file[:len(file)-4]}_{start}.txt', 'w', encoding= 'utf-8-sig') as sql:
                    # with open(os.path.join(subfolder)+ f'//{file[:len(file)-4]}_{start}.txt', 'w', encoding= 'utf-8-sig') as sql:
                    with open(os.path.join('file-SQL')+ f'//{file[:len(file)-4]}_{start}.txt', 'w', encoding= 'utf-8-sig') as sql:
                        sql.write(f'''-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben
where account = '{account}'
and ft_no in (
{list_ft}
)
and ben_acct_no is not null
;
-- ESBDATA
select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
, srcbnk.name as ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_102 <> '{account}'
and t1.t24_trans_no is not null
union all
select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
on t.trans_id = t1.serial_no
left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
where t.trans_no in (
{list_ft}
)
and t1.field_103 <> '{account}'
and t1.t24_trans_no is not null
union all
select t2.trans_no, t1.debit_account_no as tai_khoan_doi_ung, da.t_short_title as ten_doi_ung
, 'Ngân hàng TMCP Phương Đông (OCB)' as ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.debit_account_no <> '{account}'
union all
select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
where t1.payment_type='InternalPayment' and t2.trans_no in (
{list_ft}
)
and t1.credit_account <> '{account}'
;
''')
                    start_index = end_index
                    end_index = end_index + sl_row
                    start +=1
            sql.close()
            wb.close()
            app.quit()

Loading folder d:\python\bank-Statement\check-Statements-File\file-Statements\364990 file saoke_8193635-NGUYEN VAN TUC.xls......


#### ONLY ACCOUNT

In [4]:
import pandas as pd
import os
import xlwings as xw
from pathlib import  Path
path = os.getcwd()
for file_0 in os.listdir(os.path.join(path,'file-SQL')):
    os.remove(os.path.join(path,'file-SQL',file_0))
path = os.getcwd()
root_folder = 'file-Statements'
for root, dirs, files in os.walk(os.path.join(path, root_folder)):
    for dir in dirs:
        subfolder = os.path.join(root, dir)
        for file in os.listdir(subfolder):
            
            # print(os.path.join(root, dir, file))
            print(f'Loading folder {subfolder} file {file}......')

            df_sk = pd.read_excel(os.path.join(subfolder, file))
            df_sk = df_sk.loc[df_sk['ID'].str.startswith('FT')]
            df_sk = df_sk['ID'].apply(lambda x: x[:12])
            
            app = xw.App()
            wb = xw.Book(os.path.join(subfolder, file))
            # account = wb.sheets[0].range("E3").raw_value


            a = list(map(lambda x: "'"+x+"'",df_sk.values))
            list_ft = ','.join(a) #string
            count = df_sk.count()
            if count < 1000:
                # with open(f'{file[:len(file)-4]}.sql', 'w', encoding= 'utf-8-sig') as sql:
                # with open(os.path.join(subfolder)+ f'//{file[:len(file)-4]}.txt', 'w', encoding= 'utf-8-sig') as sql:
                with open(os.path.join('file-SQL')+ f'//{file[:len(file)-5]}.txt', 'w', encoding= 'utf-8-sig') as sql:
                    sql.write(f'''-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben t 
where ben_acct_no is not null
and t.ft_no in (
{list_ft}
)
;
-- ESBDATA
select distinct * from (
	select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
	, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
	from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
	inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
	inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
	where t1.payment_type='InternalPayment'
	and t2.trans_no in (
	{list_ft}
	)
	union all
	select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
	, srcbnk.name as ngan_hang_doi_ung
	from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
	on t.trans_id = t1.serial_no
	left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
	left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
	where t1.t24_trans_no is not null
	and t.trans_no in (
	{list_ft}
	)
	union all
	select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
	from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
	on t.trans_id = t1.serial_no
	left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
	left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
	where t1.t24_trans_no is not null
	and t.trans_no in (
	{list_ft}
	)
) a
;
''')
            if count >= 1000:
                print(f'Loading file > 1000 {file}......')
                sl_file = (count // 1000) + 1
                sl_row = (count // sl_file)  + 1
                start = 1
                start_index = 0
                end_index = sl_row
                while start <= sl_file:
                    new_list_a = a[start_index:end_index]
                    list_ft = ','.join(new_list_a)
                    # with open(f'{file[:len(file)-4]}_{start}.txt', 'w', encoding= 'utf-8-sig') as sql:
                    # with open(os.path.join(subfolder)+ f'//{file[:len(file)-4]}_{start}.txt', 'w', encoding= 'utf-8-sig') as sql:
                    with open(os.path.join('file-SQL')+ f'//{file[:len(file)-5]}_{start}.txt', 'w', encoding= 'utf-8-sig') as sql:
                        sql.write(f'''-- OCBDWP
select ft_no trans_no
, ben_acct_no tai_khoan_doi_ung
, ben_cust_name ten_doi_ung
, ben_bank_name ngan_hang_doi_ung
from ocbdwp.v_cust_acct_stmt_ben t 
where ben_acct_no is not null
and t.ft_no in (
{list_ft}
)
;
-- ESBDATA
select distinct * from (
	select t2.trans_no, t1.credit_account as tai_khoan_doi_ung
	, ca.t_short_title as ten_doi_ung, 'Ngân hàng TMCP Phương Đông (OCB)' ngan_hang_doi_ung
	from ESBDATA.omni_transaction_hist t1 inner join ESBDATA.transactions t2 on t1.trans_id=t2.trans_id
	inner join ESBDATA.t24_account da on t1.debit_account_no = da.id
	inner join ESBDATA.t24_account ca on t1.credit_account = ca.id
	where t1.payment_type='InternalPayment'
	and t2.trans_no in (
	{list_ft}
	)
	union all
	select t1.t24_trans_no trans_no, t1.field_102 as tai_khoan_doi_ung, t1.field_48 as ten_doi_ung
	, srcbnk.name as ngan_hang_doi_ung
	from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
	on t.trans_id = t1.serial_no
	left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
	left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
	where t1.t24_trans_no is not null
	and t.trans_no in (
	{list_ft}
	)
	union all
	select t1.t24_trans_no trans_no, t1.field_103 tai_khoan_doi_ung, t1.field_120 ten_doi_ung, dstbnk.NAME ngan_hang_doi_ung
	from esbdata.transactions t inner join ESBDATA.smartlink_transaction_trace t1 
	on t.trans_id = t1.serial_no
	left join ESBDATA.smartlink_banks_ben srcbnk on t1.field_32 = srcbnk.code
	left join esbdata.smartlink_banks_ben dstbnk on t1.field_100 = dstbnk.code
	where t1.t24_trans_no is not null
	and t.trans_no in (
	{list_ft}
	)
) a
;
''')
                    start_index = end_index
                    end_index = end_index + sl_row
                    start +=1
            sql.close()
            wb.close()
            app.quit()

Loading folder d:\python\bank-Statement\check-Statements-File\file-Statements\359221 file TKWA_2022.xlsx......


In [ ]:
lst = [0,1,2,3,4,5,6,7,8,9]
sl_file = (len(lst) // 3) + 1

sl_ele = (len(lst) // sl_file) + 1

print(sl_file)

print(sl_ele)

start = 1
start_index = 0
end_index = sl_ele 
while start < sl_file:
    

    print(start)
    print(start_index)
    print(end_index)
    print(lst[start_index: end_index])

    start +=1
    start_index = end_index
    end_index = end_index + sl_ele
print(lst[start_index:])







In [7]:
path = os.getcwd()
folder = 'file-Statements'
for root, dirs, files in os.walk(path+folder):
    for dir in dirs:
        folder = os.path.join(root, dir)
        for file in os.listdir(folder):
            
            # print(os.path.join(root, dir, file))

d:\python\filter\saoke\351518\các FT cần kiểm tra(1).xlsx
d:\python\filter\saoke\351518\các FT cần kiểm tra.xlsx
d:\python\filter\saoke\351801\saoke 0071100004578002 27.05.2020 ĐẾN 22.11.2022.xls
d:\python\filter\saoke\351801\saoke 0071105899999999 TỪ 27.05.2020 ĐẾN 22.11.2022.xls
d:\python\filter\saoke\351967\0145106299999999.xls


In [37]:
input_dir = Path.cwd()
files = list(input_dir.rglob("*.xls*"))
# wb = xw.Book(os.path.join(files[0]))

files[0]


WindowsPath('d:/python/filter/saoke/351518/các FT cần kiểm tra(1).xlsx')

In [ ]:
path = os.getcwd()
root_folder = 'file-Statements'
for root, dirs, files in os.walk(path+root_folder):
    for dir in dirs:
        subfolder = os.path.join(root, dir)
        for file in os.listdir(subfolder):
            
            # print(os.path.join(root, dir, file))
            print(f'Loading folder {subfolder} file {file}......')

            df_sk = pd.read_excel(os.path.join(subfolder, file))
df_sk

In [ ]:
path = os.getcwd()
folder1 = 'file-Statements'
folder2 = '\#folder'
list_file1 = []
for file in os.listdir(path+ folder1):
    if file.endswith('.sql'):
        # print(f'Loading file {file}......')
        list_file1.append(file[:len(file)-4])
print(len(list_file1))
print(list_file1)
for file in os.listdir(path+ folder2):
    if file.endswith('.xls'):
        if (file[:len(file)-4]) not in list_file1:
            print(file)
